In [10]:
import os
import pandas as pd
from llama_cloud_services import LlamaExtract
from schemas import FinancialPosition_2024_25  #This could be adjusted through schemas.py
from dotenv import load_dotenv

In [ ]:
PDF_ROOT = 
OUTPUT_ROOT = 
os.makedirs(OUTPUT_ROOT, exist_ok=True)  
# AGENT_ID = "6edffe95-2dac-4992-8f32-7c179c60850a" #Different based on your LLamaCloud account

In [14]:
load_dotenv() #make sure the API key is in the .env file
extractor = LlamaExtract()

#uncomment the below line if you are creating the agent for the first time
agent = extractor.create_agent(name = "financial_position-2024", data_schema=FinancialPosition_2024_25)

# agent = extractor.get_agent(id = AGENT_ID)

#uncomment the following lines if you updated the schema
# agent.data_schema = FinancialPosition_2024_25
# agent.save()

No project_id provided, fetching default project.


In [ ]:
OUTPUT_FILE = os.path.join(OUTPUT_ROOT, "all_schools.xlsx")

writer = pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl")

for school in sorted(os.listdir(PDF_ROOT)):
    school_dir = os.path.join(PDF_ROOT, school)
    if not os.path.isdir(school_dir):
        continue

    combined   = {}
    first_keys = None
    for fname in sorted(os.listdir(school_dir)):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {school}/{fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        df = pd.DataFrame.from_dict(combined, orient="index", columns=["2024-25"])
        df.index.name = "Metric"
        sheet_name = school[:31]
        df.to_excel(writer, sheet_name=sheet_name)
    else:
        print(f"No data for {school}.")

writer.close()
print(f"All schools written to {OUTPUT_FILE}")